## Neural Network Model

In [5]:
import pandas as pd
import pandas as pd
import numpy as np
from numpy import array
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

nfl = pd.read_csv("NFL_data_super_cleaned.csv")

### Data Preprocessing

In [6]:
play = array(nfl["play_type"])

play_encoder = LabelEncoder()
play_encoded =  play_encoder.fit_transform(play) 
 
play_onehot_encoder = OneHotEncoder(sparse=False)

play_encoded = play_encoded.reshape(len(play_encoded), 1) 
play_onehot_encoded = play_onehot_encoder.fit_transform(play_encoded)

nfl["play_type"] = play_onehot_encoded.tolist()

posteam = array(nfl["posteam"])

posteam_encoder = LabelEncoder()
posteam_encoded =  posteam_encoder.fit_transform(posteam) 
 
posteam_onehot_encoder = OneHotEncoder(sparse=False)
posteam_encoded = posteam_encoded.reshape(len(posteam_encoded), 1) 
posteam_onehot_encoded = pd.DataFrame(posteam_onehot_encoder.fit_transform(posteam_encoded))

nfl1 = nfl.drop(columns = ['posteam'])
nfl2 = posteam_onehot_encoded.join(nfl1)
nfl2.columns = ['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL', 'DEN', 'DET', 
'GB', 'HOU', 'IND', 'JAX', 'KC', 'LAC', 'LAR', 'LV', 'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 
'PHI', 'PIT', 'SEA', 'SF', 'TB', 'TEN', 'WAS', 'yardline_100', 'quarter_seconds_remaining', 
'qtr', 'down', 'goal_to_go', 'ydstogo', 'play_type', 'score_margin']

X = nfl2.drop("play_type", axis = 1)
y = play_onehot_encoded
X = (X-X.min())/(X.max()-X.min())
print(X)

X = X.fillna(0)

### Training and Testing the Neural Network

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

clf = MLPClassifier(solver = 'sgd', random_state = 42, activation = 'logistic', learning_rate_init = 0.3, batch_size = 100, hidden_layer_sizes = (12, 3), max_iter = 500)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test) 

prob = clf.predict_proba(X_test)
n = 0
for idx, x in enumerate(prob):
    for idx, y in enumerate(x):
        if y == max(x):
            x[idx] = 1
        else:
            x[idx] = 0
newy_pred = prob

print(confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1)))

print("Accuracy:", clf.score(X_test, y_test))

mse = mean_squared_error(y_test, newy_pred)
print('MSE:', mse)

print(classification_report(y_test, newy_pred))